# Data

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtest.utils import run_strategy, run_wfo
from backbone.utils.general_purpose import calculate_units_size, diff_pips

import itertools
import numpy as np
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
from pandas import Timestamp
import numpy as np
import random
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta

random.seed(42)

In [3]:
INITIAL_CASH = 10_000
MARGIN = 1/30
COMMISSION = 7e-4

timezone = pytz.timezone("Etc/UTC")
date_from_get_data = datetime(2021, 10, 1, tzinfo=timezone)
date_to_get_data = datetime(2024, 9, 1, tzinfo=timezone)

#
limited_testing_start_date = Timestamp('2022-01-01 00:00:00', tz='UTC')
limited_testing_end_date = Timestamp('2024-09-01 00:00:00', tz='UTC')

In [4]:

groups = [
    'Forex_Indicator',
    'CryptoCross_grp',
    'Crypto_group',
    'Energies_group',
    'Forex_group',
    'Indices_group',
    'Stocks_group'
]

In [5]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

symbols = mt5.symbols_get()

tickers = [symbol.path.split('\\')[1] for symbol in symbols if ('Stocks_group' in symbol.path or 'Energies_group' in symbol.path or 'Indices_group' in symbol.path)]

print(tickers)

intervals = [
    mt5.TIMEFRAME_H4,
    mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_H1,
    mt5.TIMEFRAME_M15,
]

symbols = get_data(tickers, intervals, date_from_get_data, date_to_get_data)

['UKOILm', 'USOILm', 'AUS200m', 'DE30m', 'FR40m', 'HK50m', 'JP225m', 'STOXX50m', 'UK100m', 'US30m', 'US500m', 'USTECm', 'AAPLm', 'ABBVm', 'ABTm', 'ADBEm', 'ADPm', 'AMDm', 'AMGNm', 'AMTm', 'AMZNm', 'ATVIm', 'AVGOm', 'BAm', 'BABAm', 'BACm', 'BIIBm', 'BMYm', 'Cm', 'CHTRm', 'CMCSAm', 'CMEm', 'COSTm', 'CSCOm', 'CSXm', 'CVSm', 'EAm', 'EBAYm', 'EQIXm', 'Fm', 'FBm', 'GILDm', 'GOOGLm', 'HDm', 'IBMm', 'INTCm', 'INTUm', 'ISRGm', 'JNJm', 'JPMm', 'KOm', 'LINm', 'LLYm', 'LMTm', 'MAm', 'MCDm', 'MDLZm', 'MMMm', 'MOm', 'MRKm', 'MSm', 'MSFTm', 'NFLXm', 'NKEm', 'NVDAm', 'ORCLm', 'PEPm', 'PFEm', 'PGm', 'PMm', 'PYPLm', 'REGNm', 'SBUXm', 'Tm', 'TMOm', 'TMUSm', 'TSLAm', 'UNHm', 'UPSm', 'Vm', 'VRTXm', 'VZm', 'WFCm', 'WMTm', 'XOMm', 'AMCm', 'BBm', 'BBBYm', 'BEKEm', 'BIDUm', 'BILIm', 'BRQSm', 'BYNDm', 'CANm', 'EDUm', 'FTNTm', 'FUTUm', 'IQm', 'JDm', 'LIm', 'NIOm', 'NTESm', 'PDDm', 'RLXm', 'TALm', 'TIGRm', 'TMEm', 'TSMm', 'VIPSm', 'XPEVm', 'YUMCm', 'ZTOm', 'XNGUSDm', 'METAm', 'IN50m']
MetaTrader5 package author: 

In [6]:
max_start_date = None
intervals_start_dates = {}

tickers = np.unique(list(symbols.keys())).tolist()

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date


intervals_start_dates


print(intervals_start_dates)
print(limited_testing_start_date)
print(limited_testing_end_date)

{16388: Timestamp('2022-03-20 20:00:00+0000', tz='UTC'), 16386: Timestamp('2022-03-20 22:00:00+0000', tz='UTC'), 16385: Timestamp('2022-03-20 22:00:00+0000', tz='UTC'), 15: Timestamp('2022-03-20 22:15:00+0000', tz='UTC')}
2022-01-01 00:00:00+00:00
2024-09-01 00:00:00+00:00


# Estrategia

In [7]:
from backbone.trader_bot import TraderBot

class BPercent(Strategy):
    pip_value = None
    minimum_units = None
    minimum_fraction = None
        
    risk= 1
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200
    b_open_threshold = 0.90
    b_close_threshold = 0.5
    atr_multiplier = 1.8

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )
        
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)
        
    def next(self):
        actual_close = self.data.Close[-1]
        b_percent = (actual_close - self.lower_band[-1]) / (self.upper_band[-1] - self.lower_band[-1])
        
        if self.position:
            if self.position.is_long:
                if b_percent >= self.b_close_threshold:
                    self.position.close()

            if self.position.is_short:
                if b_percent <= 1 - self.b_close_threshold:
                    self.position.close()

        else:
            if b_percent <= 1 - self.b_open_threshold and actual_close > self.sma[-1]:

                sl_price = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                units = self.minimum_units if units < self.minimum_units else units
                
                self.buy(
                    size=units,
                    sl=sl_price
                )
                
            if b_percent >= self.b_open_threshold and actual_close < self.sma[-1]:
                sl_price = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                units = self.minimum_units if units < self.minimum_units else units

                self.sell(
                    size=units,
                    sl=sl_price    
                )
                
    def next_live(self, trader:TraderBot):
            actual_close = self.data.Close[-1]
            b_percent = (actual_close - self.lower_band[-1]) / (self.upper_band[-1] - self.lower_band[-1])
            
            open_positions = trader.get_open_positions()
            
            if open_positions:
                if open_positions[-1].type == mt5.ORDER_TYPE_BUY:
                    if b_percent >= self.b_close_threshold:
                        trader.close_order(open_positions[-1])

                if open_positions[-1].type == mt5.ORDER_TYPE_SELL:
                    if b_percent <= 1 - self.b_close_threshold:
                        trader.close_order(open_positions[-1])

            else:

                if b_percent <= 1 - self.b_open_threshold and actual_close > self.sma[-1]:
                    
                    info_tick = trader.get_info_tick()
                    price = info_tick.ask
                    
                    sl_price = price - self.atr_multiplier * self.atr[-1]
                    
                    pip_distance = diff_pips(
                        price, 
                        sl_price, 
                        pip_value=self.pip_value
                    )
                    
                    units = calculate_units_size(
                        account_size=trader.equity, 
                        risk_percentage=self.risk, 
                        stop_loss_pips=pip_distance, 
                        pip_value=self.pip_value
                    )
                    
                    units = self.minimum_units if units < self.minimum_units else units
                    
                    lots = units * self.minimum_fraction

                    trader.open_order(
                        type_='buy',
                        price=price,
                        size=lots, 
                        sl=sl_price
                    )             
                    
                if b_percent >= self.b_open_threshold and actual_close < self.sma[-1]:
                    info_tick = trader.get_info_tick()
                    price = info_tick.bid
                    
                    sl_price = price + self.atr_multiplier * self.atr[-1]
                    
                    pip_distance = diff_pips(
                        price, 
                        sl_price, 
                        pip_value=self.pip_value
                    )
                    
                    units = calculate_units_size(
                        account_size=trader.equity, 
                        risk_percentage=self.risk, 
                        stop_loss_pips=pip_distance, 
                        pip_value=self.pip_value
                    )
                    
                    units = self.minimum_units if units < self.minimum_units else units
                    
                    lots = units * self.minimum_fraction
                    
                    trader.open_order(
                        type_='sell',
                        price=price,
                        sl=sl_price,
                        size=lots
                    )
    
STRATEGY = BPercent

# Analisis preliminar

In [8]:
strategies = [
    STRATEGY
]

experiments = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    try:
        print(ticker, interval)
        
        prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date]

        df_stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=interval,
            commission=COMMISSION, 
            prices=prices, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN
        )

        performance = pd.concat([performance, df_stats])

    except:
        print(f'hubo un problema con {ticker} {interval}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


AAPLm 16388
AAPLm 16386
AAPLm 16385
AAPLm 15
ABBVm 16388
ABBVm 16386
ABBVm 16385
ABBVm 15
ABTm 16388
ABTm 16386
ABTm 16385
ABTm 15
ADBEm 16388
ADBEm 16386
ADBEm 16385
ADBEm 15
ADPm 16388
ADPm 16386
ADPm 16385
ADPm 15
AMDm 16388
AMDm 16386
AMDm 16385
AMDm 15
AMGNm 16388
AMGNm 16386
AMGNm 16385
AMGNm 15
AMTm 16388
AMTm 16386
AMTm 16385
AMTm 15
AMZNm 16388
AMZNm 16386
AMZNm 16385
AMZNm 15
AUS200m 16388
AUS200m 16386
AUS200m 16385
AUS200m 15
AVGOm 16388
AVGOm 16386
AVGOm 16385
AVGOm 15
BABAm 16388
BABAm 16386
BABAm 16385
BABAm 15
BACm 16388
BACm 16386
BACm 16385
BACm 15
BAm 16388
BAm 16386
BAm 16385
BAm 15
BBm 16388
BBm 16386
BBm 16385
BBm 15
BEKEm 16388
BEKEm 16386
BEKEm 16385
BEKEm 15
BIDUm 16388
BIDUm 16386
BIDUm 16385
BIDUm 15
BIIBm 16388
BIIBm 16386
BIIBm 16385
BIIBm 15
BILIm 16388
BILIm 16386
BILIm 16385
BILIm 15
BMYm 16388
BMYm 16386
BMYm 16385
BMYm 15
CHTRm 16388
CHTRm 16386
CHTRm 16385
CHTRm 15
CMCSAm 16388
CMCSAm 16386
CMCSAm 16385
CMCSAm 15
CMEm 16388
CMEm 16386
CMEm 16385
CMEm 

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backtest\utils.py:383: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backtest\utils.py:383: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


JP225m 16385


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backtest\utils.py:383: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


JP225m 15


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backtest\utils.py:383: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


JPMm 16388
JPMm 16386
JPMm 16385
JPMm 15
KOm 16388
KOm 16386
KOm 16385
KOm 15
LINm 16388
LINm 16386
LINm 16385
LINm 15
LIm 16388
LIm 16386
LIm 16385
LIm 15
LLYm 16388
LLYm 16386
LLYm 16385
LLYm 15
LMTm 16388
LMTm 16386
LMTm 16385
LMTm 15
MAm 16388
MAm 16386
MAm 16385
MAm 15
MCDm 16388
MCDm 16386
MCDm 16385
MCDm 15
MDLZm 16388
MDLZm 16386
MDLZm 16385
MDLZm 15
METAm 16388
METAm 16386
METAm 16385
METAm 15
MMMm 16388
MMMm 16386
MMMm 16385
MMMm 15
MOm 16388
MOm 16386
MOm 16385
MOm 15
MRKm 16388
MRKm 16386
MRKm 16385
MRKm 15
MSFTm 16388
MSFTm 16386
MSFTm 16385
MSFTm 15
MSm 16388
MSm 16386
MSm 16385
MSm 15
NFLXm 16388
NFLXm 16386
NFLXm 16385
NFLXm 15
NIOm 16388
NIOm 16386
NIOm 16385
NIOm 15
NKEm 16388
NKEm 16386
NKEm 16385
NKEm 15
NTESm 16388
NTESm 16386
NTESm 16385
NTESm 15
NVDAm 16388
NVDAm 16386
NVDAm 16385
NVDAm 15
ORCLm 16388
ORCLm 16386
ORCLm 16385
ORCLm 15
PDDm 16388
PDDm 16386
PDDm 16385
PDDm 15
PEPm 16388
PEPm 16386
PEPm 16385
PEPm 15
PFEm 16388
PFEm 16386
PFEm 16385
PFEm 15
PGm 1638

In [11]:

filter_performance = performance[
    # (performance['ticker'] == 'USOILm')
    (performance['return/dd']>2)
    & (performance['stability_ratio']>0.7)
].sort_values(by=['custom_metric'], ascending=[False]).drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BPercent,NTESm,16385,0.765826,37.034229,13703.422943,5.165439,293 days 16:00:00,56.250000,1.361436,64,0.655044,13.227250,13703.422943,970 days 08:00:00,7.169619,25.074486
0,BPercent,IQm,16385,0.875743,37.149498,13714.949833,5.882361,226 days 00:00:00,58.461538,1.429493,65,0.943079,14.545844,13714.949833,970 days 05:00:00,6.315406,22.614852
0,BPercent,PDDm,15,0.910539,42.111820,14211.181964,15.959916,273 days 01:00:00,42.816092,0.624288,348,0.109221,15.788409,14211.181964,970 days 08:30:00,2.638599,14.538264
0,BPercent,TSMm,16385,0.814297,27.281972,12728.197181,8.111984,441 days 00:00:00,50.943396,0.882128,106,0.298008,16.255487,12728.197181,970 days 08:00:00,3.363169,13.990804
0,BPercent,ABBVm,16385,0.908146,18.352959,11835.295867,4.801991,285 days 19:00:00,53.968254,0.987904,63,0.295953,14.074550,11835.295867,970 days 05:00:00,3.821948,13.155451
0,BPercent,ZTOm,15,0.702199,60.036089,16003.608876,25.727038,246 days 02:15:00,45.783133,0.713646,249,0.188255,17.928717,16003.608876,970 days 05:00:00,2.333579,12.402680
0,BPercent,NVDAm,16385,0.732878,33.500585,13350.058483,12.452812,359 days 06:00:00,45.454545,0.703354,110,0.494156,16.666667,13350.058483,970 days 08:00:00,2.690202,11.727809
0,BPercent,BMYm,16386,0.875838,13.151038,11315.103792,3.785908,226 days 16:00:00,61.111111,0.998356,36,0.601241,14.623172,11315.103792,970 days 04:00:00,3.473682,9.922323
0,BPercent,CHTRm,16388,0.784173,13.390118,11339.011790,3.491522,194 days 20:00:00,52.173913,0.911046,23,1.520141,17.052767,11339.011790,970 days 04:00:00,3.835037,9.474408
0,BPercent,EDUm,16386,0.775067,13.003262,11300.326186,3.714153,178 days 00:00:00,46.153846,0.686494,26,1.306808,10.761155,11300.326186,970 days 04:00:00,3.501004,9.091057


In [10]:
0/0

ZeroDivisionError: division by zero

In [ ]:
for ticker, interval in zip(portfolio, intervals):
    
    prices = symbols[ticker][interval]

    
    df_stats = run_strategy(
        strategy=strategy,
        ticker=ticker,
        interval=interval,
        commission=COMMISSION, 
        prices=prices, 
        initial_cash=INITIAL_CASH, 
        margin=MARGIN,
        plot=True
    )
    break


# WFO

In [12]:
from backtest.utils import optim_func_2

lookback_bars = 1800
validation_bars = 180
warmup_bars = 200

all_wfo_performances = pd.DataFrame()
stats_per_symbol = {}

params = {
    'b_open_threshold' : [0.9, 0.95, 1, 1.5],
    'b_close_threshold': [0.2, 0.5, 0.6, 0.8, 0.9, 1],
}

for index, row in filter_performance.iterrows():
    
    # try:
    ticker = row.ticker
    interval = row.interval
    
    print(ticker, interval)
    
    prices = symbols[ticker][interval]

    wfo_stats, df_stats = run_wfo(
        strategy=STRATEGY,
        ticker=ticker,
        interval=interval,
        prices=prices,
        initial_cash=INITIAL_CASH,
        commission=COMMISSION,
        margin=MARGIN,
        optim_func=optim_func_2,
        params=params,
        lookback_bars=lookback_bars,
        warmup_bars=warmup_bars,
        validation_bars=validation_bars
    )
    
    if ticker not in stats_per_symbol.keys():
        stats_per_symbol[ticker] = {}

    stats_per_symbol[ticker][interval] = wfo_stats

    all_wfo_performances = pd.concat([all_wfo_performances, df_stats])
    
    # except:
    #     print(f'No se pudo ejecutar para el ticker {ticker}')

all_wfo_performances['return/dd'] = all_wfo_performances['return'] / -all_wfo_performances['drawdown']
all_wfo_performances['drawdown'] = -all_wfo_performances['drawdown']
all_wfo_performances['custom_metric'] = (all_wfo_performances['return'] / (1 + all_wfo_performances.drawdown)) * np.log(1 + all_wfo_performances.trades)

all_wfo_performances.drawdown_duration = pd.to_timedelta(all_wfo_performances.drawdown_duration)
all_wfo_performances.drawdown_duration = all_wfo_performances.drawdown_duration.dt.days

all_wfo_performances.sort_values(by='return/dd', ascending=False)

NTESm 16385


IQm 16385


PDDm 15


KeyboardInterrupt: 

In [ ]:
all_wfo_performances[
    (all_wfo_performances['return/dd'] > 1)
    
].sort_values(by='custom_metric', ascending=False)

# Montecarlo

In [ ]:
filtered_wfo_performance = all_wfo_performances[
    (all_wfo_performances['stability_ratio'] > 0.7)
].sort_values(by='return/dd', ascending=False)

filtered_wfo_performance

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from backtest.utils import montecarlo_statistics_simulation

# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []
montecarlo_simulations = {}

all_drawdowns = pd.DataFrame()
all_returns = pd.DataFrame()

for index, row in filtered_wfo_performance.iterrows():
    ticker = row.ticker
    interval = row.interval
    
    print(f"Procesando ticker: {ticker}")
    trades_history = stats_per_symbol[ticker][interval]._trades
    eq_curve = stats_per_symbol[ticker][interval]._equity_curve
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, synthetic_drawdown_curve, synthetic_return_curve = montecarlo_statistics_simulation(
        equity_curve=eq_curve,
        trade_history=trades_history, 
        n_simulations=100_000, 
        initial_equity=INITIAL_CASH, 
        threshold_ruin=0.8, 
        return_raw_curves=True,
        percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
    )
    
    montecarlo_simulations[ticker] = mc
    
    synthetic_drawdown_curve['ticker'] = ticker
    synthetic_return_curve['ticker'] = ticker
    
    all_drawdowns = pd.concat([all_drawdowns, synthetic_drawdown_curve])
    all_returns = pd.concat([all_returns, synthetic_return_curve])

In [ ]:
dd_df = pd.DataFrame()
returns_df = pd.DataFrame()

for ticker, mc in montecarlo_simulations.items():
    mc = mc.rename(
        columns={
            'Drawdown (%)': f'drawdown_{ticker}',
            'Final Return (%)': f'return_{ticker}',
        }
    )

    if dd_df.empty:
        dd_df = mc[[f'drawdown_{ticker}']]
    
    else:
        dd_df = pd.merge(
            dd_df,
            mc[[f'drawdown_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
    if returns_df.empty:
        returns_df = mc[[f'return_{ticker}']]
    
    else:
        returns_df = pd.merge(
            returns_df,
            mc[[f'return_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
display(dd_df)
display(returns_df)


In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 18))
sns.boxplot(data=all_drawdowns, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = all_drawdowns["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = all_drawdowns["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Generar el gráfico
plt.figure(figsize=(12, 20))
sns.boxplot(data=all_returns, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = all_returns["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = all_returns["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
